# Facility Sizing Problem

Consider the Facility Sizing Problem described in Chapter 4 of the Benders Decomposition lecture notes.
The data of an instance of the problem is provided below, together with a class that represents the problem and a class that represents the full (non-decomposed) model.
- Q1. Solve the instance of the problem using Benders decomposition
- Q2. Ensure that the solution obtained is the same as the one give by the non-decomposed model.

## Data of an instance

In [1]:
import random as r
n_locations = 5
n_customers = 5

r.seed(1)
# Random capacity costs between 100 and 300
fixed_costs = [(100 + (r.random() * 200)) for i in range(n_locations)]

# Random delivery costs between 10 and 40
delivery_costs = {(i,j):(10 + r.random() * 30) for j in range(n_customers) for i in range(n_locations)}

# Random demands between 50 and 100
demands = [(50 + (r.random() * 50)) for j in range(n_customers)]

# Random capacities between 100 and 140
capacities = [(120 + (r.random() * 20)) for i in range(n_locations)]


## Class for the Facility Sizing Problem

In [2]:
class FacilitySizingProblem:

    def __init__(self,n_facilities:int,n_customers:int,fixed_costs:list,delivery_costs:dict,demands:list,capacity:list):
        self.n_facilities = n_facilities
        self.n_customers = n_customers
        self.fixed_costs = fixed_costs
        self.delivery_costs = delivery_costs
        self.demands = demands
        self.capacity = capacity
        


In [3]:
p = FacilitySizingProblem(n_locations, n_customers, fixed_costs, delivery_costs, demands, capacities)

## Class for the full model

In [4]:
from gurobipy import Model, GRB, quicksum

In [5]:
class FullModel:

    def __init__(self, fsp:FacilitySizingProblem):
        self.fsp = fsp
        self.m = Model()

        # Creates the variables
        self.y = self.m.addVars(fsp.n_facilities, fsp.n_customers, name="y")
        self.x = self.m.addVars(fsp.n_facilities, name="x")

        term1 = self.x.prod(fsp.fixed_costs)
        term2 = self.y.prod(fsp.delivery_costs)
        self.m.setObjective(term1+term2, GRB.MINIMIZE)

        # Constraints

        self.m.addConstrs(self.y.sum(i, '*') <= self.x[i] for i in range(fsp.n_facilities))
        self.m.addConstrs(self.y.sum('*', j) >= fsp.demands[j] for j in range(fsp.n_customers))
        self.m.addConstrs(self.x[i] <= self.fsp.capacity[i] for i in range(fsp.n_facilities))
    

    def solve(self):
        self.m.optimize()

    def print_solution(self):
        for i in range(self.fsp.n_facilities):
            print('%s %g' % (self.x[i].varName, self.x[i].x))
        print('Obj: %g' % self.m.objVal)


# Solution

## Q1: Benders decomposition
We start by implementing the feasibility subproblem

In [6]:

class FSP:

    def __init__(self, fsp:FacilitySizingProblem, x:list):
        self.fsp = fsp
        self.m = Model()

        # Creates the variables
        y = self.m.addVars(fsp.n_facilities, fsp.n_customers, name="y")
        v1 = self.m.addVars(fsp.n_facilities, name="v1")
        v2 = self.m.addVars(fsp.n_customers, name="v2")

        # Creates the objective
        expr = v1.sum() + v2.sum()
        self.m.setObjective(expr, GRB.MINIMIZE)

        # Constraints
        self.demand_c = self.m.addConstrs(y.sum('*', j) + v1[j] >= fsp.demands[j] for j in range(fsp.n_customers))
        self.capacity_c = self.m.addConstrs(y.sum(i, '*') - v2[i] <= x[i] for i in range(fsp.n_facilities))        

    def solve(self):
        self.m.optimize()

    def getResults(self):
        '''
        Returns, in this order,
        1) the objective value,
        2) the vector of optimal duals for the capacity constraints
        3) the vector of optimal duals for the demand constraints
        Alternatively, one may create three methods, one for the objective value, one for the duals of the capacity constraints
        and one for the duals of the demand constraints.
        '''
        dualsCC = [self.capacity_c[i].Pi for i in range(self.fsp.n_facilities)]
        dualsDC = [self.demand_c[j].Pi for j in range(self.fsp.n_customers)]

        return self.m.objVal, dualsCC, dualsDC


Optimality subproblem

In [7]:
class OSP:

    def __init__(self, fsp:FacilitySizingProblem, x:list):
        self.fsp = fsp
        self.m = Model()

        # Creates the variables
        y = self.m.addVars(fsp.n_facilities, fsp.n_customers, name="y")

        # Creates the objective
        self.m.setObjective(y.prod(fsp.delivery_costs), GRB.MINIMIZE)

        # Constraints
        self.capacity_constr = self.m.addConstrs(y.sum(i, '*') <= x[i] for i in range(self.fsp.n_facilities))
        self.demand_constr = self.m.addConstrs(y.sum('*', j) >= fsp.demands[j] for j in range(fsp.n_customers))        

    def solve(self):
        self.m.optimize()

    def getResults(self):
        '''
        Returns, in this order,
        1) the objective value,
        2) the vector of optimal duals for the capacity constraints
        3) the vector of optimal duals for the demand constraints
        Alternatively, one may create three methods, one for the objective value, one for the duals of the capacity constraints
        and one for the duals of the demand constraints.
        '''
        print(self.capacity_constr)
        dualsCC = self.m.getAttr(GRB.Attr.Pi, self.capacity_constr)
        dualsDC = self.m.getAttr(GRB.Attr.Pi, self.demand_constr)

        return self.m.objVal, dualsCC, dualsDC


Master problem

In [8]:
class Master:

    def __init__(self, fsp:FacilitySizingProblem):
        self.fsp = fsp
        self.m = Model()

        # Creates the variables
        self.x = self.m.addVars(fsp.n_facilities, vtype=GRB.CONTINUOUS, name="x")
        self.phi = self.m.addVar(name="phi")

        # Creates the objective
        expr = self.phi + quicksum([self.fsp.fixed_costs[i] * self.x[i] for i in range(self.fsp.n_facilities)])
        self.m.setObjective(expr, GRB.MINIMIZE)

        # Creates the constraints
        self.m.addConstrs(self.x[i] <= self.fsp.capacity[i] for i in range(fsp.n_facilities))

    def solve(self):
        self.m.optimize()

    def get_solution(self):
        '''
        Returns two thing:
        - the x solution (a list of values, one for each facility)
        - the phi solution (a scalar).
        Alternatively, one may create two methods, one that returns only x and one that returns only phi.
        '''
        return [self.x[i].x for i in range(self.fsp.n_facilities)], self.phi.x

    def add_feasibility_cut(self,dualsCC:list,dualsDC:list):
        '''
        Adds a feasibility cut given the duals of the feasibility subproblem (extreme rays of polyhedral cone V).
        '''
        self.m.addConstr(quicksum([dualsCC[i] * self.x[i] for i in range(self.fsp.n_facilities)])
                         <= - sum([dualsDC[j] * self.fsp.demands[j] for j in range(self.fsp.n_customers)]))

        print("Added feasibility cut")

    def add_optimality_cut(self,dualsCC:list,dualsDC:list):
        '''
        Adds an optimality cut given the duals of the optimality subproblem (extreme points of the dual of the optimality subproblem).
        '''
        self.m.addConstr(self.phi - quicksum([dualsCC[i] * self.x[i] for i in range(self.fsp.n_facilities)])
                         >= sum([dualsDC[j] * self.fsp.demands[j] for j in range(self.fsp.n_customers)]))
        print("Added optimality cut")

    def print_solution(self):
        for i in range(self.fsp.n_facilities):
            print('%s %g' % (self.x[i].varName, self.x[i].x))
        print('Obj: %g' % self.m.objVal)





We can finally put everything together and implement the Benders decomposition algorithm

In [9]:
mp = Master(p)
solved = False
while not solved:
    # Solve MP
    mp.solve()

    # Get the x solution
    x, phi = mp.get_solution()

    # Check feasibility by solving the fsp
    feasibility_sp = FSP(p,x)
    feasibility_sp.solve()
    fsp_obj, fsp_dualsCC, fsp_dualsDC = feasibility_sp.getResults()
    print(fsp_dualsCC)
    print("FSP obj ", fsp_obj)
    if fsp_obj > 0:
        # In this case we need a feasibility cut
        print("Adding a feasibility cut")
        mp.add_feasibility_cut(fsp_dualsCC,fsp_dualsDC)
    else:
        print("Checking optimality")
        # In this case we can check optimality
        optimality_sp = OSP(p,x)
        optimality_sp.solve()
        osp_obj, osp_dualsCC, osp_dualsDC = optimality_sp.getResults()
        print("Phi = ",phi)
        print("Obj = ", osp_obj)
        if phi >= osp_obj - 1e-6 :
            print("Problem solved")
            solved = True
        else:
            # In this case we add an optimality cut
            mp.add_optimality_cut(osp_dualsCC, osp_dualsDC)
            
print("Solution BD")
mp.print_solution()

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 128 physical cores, 128 logical processors, using up to 32 threads
Optimize a model with 5 rows, 6 columns and 5 nonzeros
Model fingerprint: 0x0f5eca41
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+02, 1e+02]
Presolve removed 5 rows and 6 columns
Presolve time: 0.04s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.05 seconds (0.00 work units)
Optimal objective  0.000000000e+00
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 128 physical cores, 128 logical processors, using up to 32 threads
Optimize a model with 10 rows, 35 columns and 60 nonzeros
Model fingerprint: 0x72064f5f
C

## Q2. Full Model

In [10]:
m = FullModel(p)
m.solve()
print("Solution Original Problem")
m.print_solution()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 128 physical cores, 128 logical processors, using up to 32 threads
Optimize a model with 15 rows, 30 columns and 60 nonzeros
Model fingerprint: 0x761d71f7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+01, 1e+02]
Presolve removed 5 rows and 0 columns
Presolve time: 0.01s
Presolved: 10 rows, 30 columns, 55 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.020073e+01   0.000000e+00      0s
      12    5.3259865e+04   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.325986470e+04
Solution Original Problem
x[0] 120.43
x[1] 0
x[2] 0
x[3] 132.846
x[4] 68.3302
Obj: 53259.9


The two solutions are the same, so we can trust that our implementation is correct.